#통합 책 추천 시스템

- 고정 userid = 88705 / 254321 / 182459 / 161936 / 226482 / random 1 : 121184/ random 2 -> 204167
/ 제일 부정적인 사용자
- 10가지의 방식을 활용하여 user에게 추천한다.

1. 베스트셀러/작가 - 허재훈
1. 베스트셀러/지역 - 백경관
1. 콘텐츠베이스 - 책 제목 - 송도훈
1. 협력필터링 / KNN-cosine 3가지 방식 - 허재훈
1. 행렬분해 / SVD latent 150 - 허재훈
1. 행렬분해 / NMF - 송도훈
1. 행렬분해 / ALS - 허재훈

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',
                          sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_rating.drop(index = book_rating[book_rating['Book-Rating'] == 0].index, inplace = True) #평점 0점은 결측치 처리
book_rating

<ipython-input-3-e9cc15fc3536>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',


,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
                        sep = ';', encoding = 'latin_1', error_bad_lines = False)
# 불필요한 URL, 출판사 값 버리기
book_info.drop(columns = ['Image-URL-S','Image-URL-M', 'Image-URL-L', 'Publisher'], inplace = True)
book_info

<ipython-input-4-9cc67933f6d9>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expec

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271356,0525447644,From One to One Hundred,Teri Sloat,1991
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271358,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
# 결측치가 매우 소수의 행에 존재하므로 결측치가 있는 행만 제거 - 도훈님 코드
book_info.dropna(inplace=True)
book_info.reset_index(drop=True, inplace=True)
book_info

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271355,0525447644,From One to One Hundred,Teri Sloat,1991
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271357,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_user

<ipython-input-6-e195780cffae>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


##베스트 셀러 작가별

In [ ]:
#작가의 결측치 확인
book_info['Book-Author'].isna().value_counts()

False    271359
Name: Book-Author, dtype: int64

In [ ]:
book_rating_info = pd.merge(book_rating, book_info[['ISBN', 'Book-Title','Book-Author']], on = 'ISBN')
book_rating_info

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
0,276726,0155061224,5,Rites of Passage,Judith Rae
1,276729,052165615X,3,Help!: Level 1,Philip Prowse
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather
3,276744,038550120X,7,A Painted House,JOHN GRISHAM
4,11676,038550120X,10,A Painted House,JOHN GRISHAM
...,...,...,...,...,...
383836,276688,0425156737,2,Graven Images,Jane Waterhouse
383837,276688,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries),Sally Gunning
383838,276688,1575660792,7,Gray Matter,Shirley Kennett
383839,276704,0380796155,5,White Abacus,Damien Broderick


###타겟 유저

In [ ]:
# 타겟 유저 설정
target_user_id = 161936

In [ ]:
book_rating_info[book_rating_info['User-ID'] == target_user_id]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
7893,161936,0452282152,9,Girl with a Pearl Earring,Tracy Chevalier
39630,161936,0451526341,8,Animal Farm,George Orwell
42017,161936,1558743669,10,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
48087,161936,0099771519,9,Memoirs of a Geisha Uk,Arthur Golden
49549,161936,0552998001,9,Blackberry Wine,Joanne Harris
53473,161936,0142001430,9,Year of Wonders,Geraldine Brooks
74187,161936,075285111X,7,A Question of Blood,Ian Rankin
78993,161936,068484477X,9,STONES FROM THE RIVER,Ursula Hegi
84143,161936,0449213447,8,The Chosen,Chaim Potok
91660,161936,0385475772,8,Nine Parts of Desire: The Hidden World of Isla...,Geraldine Brooks


In [ ]:
target_user_df = book_rating_info[book_rating_info['User-ID'] == target_user_id]
target_user_top3 = target_user_df.sort_values('Book-Rating', ascending = False).head(3)
target_user_top3

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
42017,161936,1558743669,10,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
335402,161936,058609119X,10,Five,Doris Lessing
335394,161936,0140172505,10,Tell Me I'm Here,Anne Deveson


In [ ]:
top3_author = list(target_user_top3['Book-Author'])
top3_author

['Dave Pelzer', 'Doris Lessing', 'Anne Deveson']

In [ ]:
book_rating_top3_author = book_rating_info[book_rating_info['Book-Author'].isin(top3_author)]

In [ ]:
# book_rating_top3_author['Book-Author'] = pd.Categorical(book_rating_top3_author['Book-Author'],
#                                                  categories=top3_author, ordered=True)
# book_rating_top3_author.sort_values('Book-Author', inplace=True)
# book_rating_top3_author

In [ ]:
count_by_author = book_rating_top3_author.groupby('ISBN').agg('count').sort_values(
    'Book-Rating', ascending = False)['Book-Rating']
count_by_author[:10]

ISBN
1558743669    134
1558745157     70
0752837613     12
0525947698      5
0739400614      4
006093056X      4
0586068805      4
0586053107      3
1402505272      2
0006550835      2
Name: Book-Rating, dtype: int64

In [ ]:
# 책 제목도 df에 추가하기
count_bs_rec = pd.merge(count_by_author, book_info[['ISBN', 'Book-Title','Book-Author']], on = 'ISBN')
count_bs_rec[:10]

,ISBN,Book-Rating,Book-Title,Book-Author
0,1558743669,134,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
1,1558745157,70,The Lost Boy: A Foster Child's Search for the ...,Dave Pelzer
2,0752837613,12,The Lost Boy,Dave Pelzer
3,0525947698,5,The Privilege of Youth: A Teenager's Story of ...,Dave Pelzer
4,0739400614,4,A Child's Called It and The Lost Boy,Dave Pelzer
5,006093056X,4,Mara and Dann : An Adventure,Doris Lessing
6,0586068805,4,Good Terrorist,Doris Lessing
7,0586053107,3,Shikasta Re Colonised Planet 5,Doris Lessing
8,1402505272,2,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
9,0006550835,2,Mara and Dann,Doris Lessing


In [ ]:
#빈도 기반 추천 - 나와 유사한 사람들이 가장 많이 본 책
avg_by_author = book_rating_top3_author.groupby('ISBN').agg('mean').sort_values(
    'Book-Rating', ascending = False)['Book-Rating']
avg_by_author[:10]

ISBN
058609119X    10.00
3596252024    10.00
3442730007    10.00
1568951485    10.00
0865380805    10.00
0140172505    10.00
0613173538    10.00
0739400614     9.25
3455019072     9.00
3442720753     9.00
Name: Book-Rating, dtype: float64

In [ ]:
avg_bs_rec = pd.merge(avg_by_author, book_info[['ISBN', 'Book-Title','Book-Author']], on = 'ISBN')
avg_bs_rec[:10]

,ISBN,Book-Rating,Book-Title,Book-Author
0,058609119X,10.00,Five,Doris Lessing
1,3596252024,10.00,Die Memoiren einer Ã?Â?berlebenden.,Doris Lessing
2,3442730007,10.00,Der Wind weht unsere Worte fort. Afghanische B...,Doris Lessing
3,1568951485,10.00,A Man Named Dave (Wheeler Large Print Press (l...,Dave Pelzer
4,0865380805,10.00,Doris Lessing: Conversations (Ontario Review P...,Doris Lessing
5,0140172505,10.00,Tell Me I'm Here,Anne Deveson
6,0613173538,10.00,The Lost Boy: A Foster Child's Search for the ...,Dave Pelzer
7,0739400614,9.25,A Child's Called It and The Lost Boy,Dave Pelzer
8,3455019072,9.00,Das fÃ?Â¼nfte Kind.,Doris Lessing
9,3442720753,9.00,Das fÃ?Â¼nfte Kind.,Doris Lessing


In [ ]:
result_bs_df = pd.concat([count_by_author, avg_by_author], axis=1)
result_bs_df.columns = ['Count', 'Average']
result_bs_df = pd.merge(result_bs_df, book_info[['ISBN','Book-Title','Book-Author']], on = 'ISBN') #제목 추가
result_bs_df.head(10) #탑 10

,ISBN,Count,Average,Book-Title,Book-Author
0,1558743669,134,8.000000,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
1,1558745157,70,8.200000,The Lost Boy: A Foster Child's Search for the ...,Dave Pelzer
2,0752837613,12,7.750000,The Lost Boy,Dave Pelzer
3,0525947698,5,8.800000,The Privilege of Youth: A Teenager's Story of ...,Dave Pelzer
4,0739400614,4,9.250000,A Child's Called It and The Lost Boy,Dave Pelzer
5,006093056X,4,5.500000,Mara and Dann : An Adventure,Doris Lessing
6,0586068805,4,6.250000,Good Terrorist,Doris Lessing
7,0586053107,3,7.333333,Shikasta Re Colonised Planet 5,Doris Lessing
8,1402505272,2,5.500000,"A Child Called \It\"": One Child's Courage to S...",Dave Pelzer
9,0006550835,2,8.000000,Mara and Dann,Doris Lessing


### 결과값 도출하기

In [ ]:
target_user_id #타겟 유저 재확인

161936

In [ ]:
# 데이터에 중복되는 제목이나 다른 ISBN을 갖고 있기에 해당 값을 제외한 추천값 출력
unseen_bs_book = set(result_bs_df['Book-Title'].to_list()) - set(target_user_df['Book-Title'].to_list())
result_bs_rec_df = result_bs_df[result_bs_df['Book-Title'].isin(unseen_bs_book)].sort_values(['Count','Average'], ascending = False)
result_bs_rec_df.head()

,ISBN,Count,Average,Book-Title,Book-Author
1,1558745157,70,8.20,The Lost Boy: A Foster Child's Search for the ...,Dave Pelzer
2,0752837613,12,7.75,The Lost Boy,Dave Pelzer
3,0525947698,5,8.80,The Privilege of Youth: A Teenager's Story of ...,Dave Pelzer
4,0739400614,4,9.25,A Child's Called It and The Lost Boy,Dave Pelzer
6,0586068805,4,6.25,Good Terrorist,Doris Lessing


In [ ]:
result_bs_rec_df.drop(columns = ['Count','Average'])[:5]

,ISBN,Book-Title,Book-Author
1,1558745157,The Lost Boy: A Foster Child's Search for the ...,Dave Pelzer
2,0752837613,The Lost Boy,Dave Pelzer
3,0525947698,The Privilege of Youth: A Teenager's Story of ...,Dave Pelzer
4,0739400614,A Child's Called It and The Lost Boy,Dave Pelzer
6,0586068805,Good Terrorist,Doris Lessing
